<img src="data/photutils_banner.svg" width=500 alt="Photutils logo" style="margin-left: 0;">

<div class="alert alert-block alert-info">
<h2 style="margin-top: 0">In this notebook, we will cover:</h2>

- PSF photometry extracting PSFs from an input image
    - Using a simulated image of elliptical PSFs
    - Using a simulated JWST NIRCam image
</div>

# PSF photometry using an image-based PSF Model

## Preliminaries

In [ ]:
# Initial imports
import numpy as np
import matplotlib.pyplot as plt

# Change some default plotting parameters
import matplotlib as mpl
mpl.rcParams['image.origin'] = 'lower'
mpl.rcParams['image.interpolation'] = 'nearest'

# Run the %matplotlib magic command to enable inline plotting
# in the current notebook.  Choose one of these:
%matplotlib inline
# %matplotlib notebook

# Build a simulated image (without noise) with a funky elliptical Moffat-like PSF

First, let's create the PSF model.

In [ ]:
from photutils import psf
from astropy.modeling import models

psfmodel = ((models.Shift(-5) & models.Shift(2)) | 
            models.Rotation2D(-20) | 
            (models.Identity(1) & models.Scale(1.5)) | 
            models.Moffat2D(1, 0, 0, 6, 4.76))

psfmodel.bounding_box = ((-10, 10), (-10, 10))
psfmodel

Let's display this PSF model.

In [ ]:
psfim = psfmodel.render().T
plt.imshow(psfim)

Next, let's create a simulated image of sources using this PSF model.

In [ ]:
psfmodel.offset_0 = psfmodel.offset_1 = 0
if hasattr(psfmodel, 'bounding_box'):
    psfimcen = psfmodel.render()
    del psfmodel.bounding_box

rng = np.random.default_rng(seed=123)

data = np.zeros((100, 100))
amps = rng.standard_normal(100)**2
xs = rng.uniform(0, data.shape[0], size=amps.size)
ys = rng.uniform(0, data.shape[1], size=amps.size)

for x, y, amp in zip(xs, ys, amps):
    psfmodel.amplitude_3 = amp
    psfmodel.offset_1 = -x
    psfmodel.offset_0 = -y
    psfmodel.render(data)

axim = plt.imshow(data)

## Now we use  `FittableImageModel` on a *rendered* version of the PSF model with no pixel subsampling

In [ ]:
plt.imshow(psfimcen)
plt.colorbar()

In [ ]:
psf_im_model = psf.FittableImageModel(psfimcen, normalize=1)
psf_im_model.bounding_box = ((-10, 10), (-10, 10))
psfrendered = psf_im_model.render()
del psf_im_model.bounding_box

plt.imshow(psfrendered)
plt.colorbar()

## Now let's try doing photometry

First we need to find stars.  We'll use the DAOPhot algorithm (which at its core is the same as most other PSF photometry tools).

First we estimate the variance in the image to give us some guess as to what might be a good threshold for star-finding.

In [ ]:
from astropy.stats import SigmaClip
from photutils.background import BiweightScaleBackgroundRMS

bkg_var = BiweightScaleBackgroundRMS(sigma_clip=SigmaClip(3))(data)
bkg_var

Then we create a `DAOStarFinder` object and run that on the image.

In [ ]:
from photutils.detection import DAOStarFinder

star_finder = DAOStarFinder(threshold=bkg_var/2, fwhm=5)
found_stars = star_finder(data)

plt.imshow(data)
plt.scatter(found_stars['xcentroid'], found_stars['ycentroid'], color='red', s=5)

found_stars

And then we create the object to do the photometry, and run it on the table of stars we found.

In [ ]:
ph = psf.BasicPSFPhotometry(psf.DAOGroup(10), None, psf_im_model, 
                            (5, 5), aperture_radius=10)

if 'xcentroid' in found_stars.colnames:
    # there's an if here simply to make sure you can run this cell
    # multiple times without re-running the star finder
    found_stars['xcentroid'].name = 'x_0'
    found_stars['ycentroid'].name = 'y_0'
    found_stars['flux'].name = 'flux_0'

res = ph.do_photometry(data, init_guesses=found_stars)


plt.imshow(data)
plt.colorbar()
plt.scatter(res['x_0'], res['y_0'], color='k')
plt.scatter(res['x_fit'], res['y_fit'], color='r', s=3, lw=0)

res

And now we try making a residual image to see how well it did.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

vmin, vmax = -0.3, 2.0

ax1.imshow(data, vmin=vmin, vmax=vmax)
ax2.imshow(ph.get_residual_image(), vmin=vmin, vmax=vmax)
ax2.scatter(res['x_fit'], res['y_fit'], color='r', s=3, lw=0);

Well that looks OK except that it looks ugly because our psf model that we fit was a bit small.  So let's try subtracting the *actual* model.

In [ ]:
subtracted_image = psf.subtract_psf(data, psf_im_model, res)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

vmin, vmax = -0.3, 2.0

ax1.imshow(data, vmin=vmin, vmax=vmax)
ax2.imshow(subtracted_image, vmin=vmin, vmax=vmax)
ax2.scatter(res['x_fit'], res['y_fit'], color='r', s=3, lw=0);

---
# Simulated JWST/NIRCam data 

Now let's try something like the above, but with simulated JWST/NIRCam data, using an oversampled PSF.  

In principal this is one of two modes one might take with real JWST data.  For many cases using a provided PSF (or generated from `webbpsf`) will be sufficient.  `photutils` also has a high-level tool to build an effective PSF (ePSF) directly from the image (see https://photutils.readthedocs.io/en/latest/epsf.html).

In [ ]:
import os

from astropy import wcs
from astropy.io import fits
from astropy.utils.data import download_file

# the simulated JWST/NIRCam data will be downloaded from STScI
base_url = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/simulated_nircam_images/nircam_f210m/'
im1_fn = 'simulated_nircam_f210m.fits'
psf1_fn = 'simulated_nircam_f210m_psf.fits'
im1_url = os.path.join(base_url, im1_fn)
psf1_url = os.path.join(base_url, psf1_fn)

im1_path = download_file(im1_url, cache=True, timeout=60)
psf1_path = download_file(psf1_url, cache=True, timeout=60)

with fits.open(im1_path) as im1f:
    im1 = im1f[1].data.copy()
    im1h = im1f[1].header.copy()
    im1wcs = wcs.WCS(im1h)

with fits.open(psf1_path) as psf1f:
    psf1 = psf1f[0].data.copy()
    psf1h = psf1f[0].header.copy()
    psf1wcs = wcs.WCS(psf1h)

In [ ]:
# this is a quick-and-easy way to rescale an image, using the 
# astropy.visualization package
from astropy.visualization import simple_norm

norm = simple_norm(im1, 'log', percent=99.0)

plt.figure(figsize=(8, 8))
plt.imshow(im1, norm=norm)

OK, let's histogram it so we can see roughly where the threshold should be.

In [ ]:
plt.hist(im1.ravel(), bins=100, histtype='step', range=(-10, 200), log=True);

In [ ]:
dsf = DAOStarFinder(100, 5)
found_stars = dsf(im1)
found_stars['xcentroid'].name = 'x_0'
found_stars['ycentroid'].name = 'y_0'
found_stars['flux'].name = 'flux_0'
found_stars

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(im1, norm=norm)
plt.scatter(found_stars['x_0'], found_stars['y_0'], lw=0, s=3, c='k')
plt.xlim(500, 1500)
plt.ylim(500, 1500);

Now we build the actual PSF model using the file that the PSF is given in. It is using external knowledge that the PSF is 5x oversampled. The simple oversampling below only works as-is because both are square, but that's true here.

In [ ]:
# note that this is *not* the same pixel scale as the image above
plt.figure(figsize=(8, 8))
psf_norm = simple_norm(psf1, 'log', percent=99.)

plt.imshow(psf1, norm=psf_norm)

psfmodel = psf.FittableImageModel(psf1, oversampling=5)

Let's now zoom in on the image somewhere and see how the model looks compared to the actual image scale.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 10))

ax1.imshow(im1, norm=norm)
ax1.set_xlim(1080, 1131)
ax1.set_ylim(1100, 1151)
ax1.set_title('Simulated image')

xg, yg = np.mgrid[-25:25, -25:25]
psf_norm2 = simple_norm(psfmodel(xg, yg), 'log', percent=99.)
ax2.imshow(psfmodel(xg, yg), norm=psf_norm2)
ax2.set_title('PSF');

Now we build a PSF photometry runner that is auto-configured to work basically the same as DAOPHOT.  All of the steps in photometry are customizable if you like, but for now we'll just use this because it's a familiar code to many people. 

In [ ]:
psfphot = psf.DAOPhotPSFPhotometry(crit_separation=5, 
                                   threshold=100, fwhm=5, 
                                   psf_model=psfmodel, fitshape=(9, 9),
                                   niters=1, aperture_radius=5)
results = psfphot(im1, init_guesses=found_stars[:100])

Now let's inspect the residual image as a whole, and zoomed in on a few particular stars.

In [ ]:
res_im1 = psfphot.get_residual_image()
plt.figure(figsize=(8, 8))

res_norm = simple_norm(res_im1, 'log', percent=99.)
plt.imshow(res_im1, norm=res_norm)
results

In [ ]:
# these *should* be one by itself and one near the core.  But you might 
# have to change the (10, 500) to something else depending on what you
# want to inspect
for i in (10, 50): 

    resulti = results[i]
    window_rad = 20

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))

    ax1.imshow(im1, norm=norm)
    ax1.set_xlim(resulti['x_fit']-window_rad, resulti['x_fit']+window_rad)
    ax1.set_ylim(resulti['y_fit']-window_rad, resulti['y_fit']+window_rad)
    ax1.scatter([resulti['x_0']], [resulti['y_0']], color='r', s=7)
    ax1.scatter([resulti['x_fit']], [resulti['y_fit']], color='w', s=7)
    ax1.set_title('Original image (star #{})'.format(i))

    ax2.imshow(res_im1, norm=res_norm)
    ax2.set_xlim(resulti['x_fit']-window_rad, resulti['x_fit']+window_rad)
    ax2.set_ylim(resulti['y_fit']-window_rad, resulti['y_fit']+window_rad)
    ax2.scatter([resulti['x_0']], [resulti['y_0']], color='r', s=7)
    ax2.scatter([resulti['x_fit']], [resulti['y_fit']], color='w', s=7)
    ax2.set_title('Subtracted image (star #{})'.format(i))

OK, looks like at least some of them worked great, but in the crowded areas more iterations/tweaks to the input parameters are needed.  See if you can tweak the parameters to make it better!

For this simulated data set we only have one band, so there's not much output "science" to show... But below you see how to get out magnitudes.

In [ ]:
# this zero-point is just a made-up number right now, 
# but it's something the instrument team will provide
inst_mag = -2.5 * np.log10(results['flux_fit'])
zero_point = 31.2  
results['cal_mag'] = zero_point + inst_mag

# if you scroll to the right you'll see the new column
results

## Exercises

As you can see, particularly for brighter stars, the above procedure doesn't do well, primarily because the star finder doesn't find them efficiently along with the faint ones.  Try manually editing the `found_stars` table and by-hand insert a few bright stars.  See if you can get them to subtract well. 

Now try playing around with the various options and see if you can do better *automatically*.